In [16]:
import sys
import os
import yaml
from pathlib import Path

# Add project root to Python path
notebook_dir = Path().absolute()  # Gets the current directory
project_root = str(notebook_dir.parent)  # Go up one level to project root
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
# Get config path
config_path = os.path.join(project_root, "config", "config.yml")
print("Project root:", project_root)
print("Config path:", config_path)

from sciagents.agents.chat_agent import ChatAgent
from sciagents.agents.message import AgentInput, Message, Role
from sciagents.tools import function_tool

Project root: c:\Users\ps\Documents\Academic\Papers\Agents\Auto_research\SciAgents
Config path: c:\Users\ps\Documents\Academic\Papers\Agents\Auto_research\SciAgents\config\config.yml


In [17]:
@function_tool(name_override="fetch_weather")
async def fetch_weather(location: str) -> str:
    return f"Sunny in {location}"

def get_stock_price(symbol: str) -> float:
    return 100.2

In [18]:
# 读取配置文件
with open(config_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

chat_agent_config = config["agents"]["ChatAgent"]

print("ChatAgent config:", chat_agent_config)


# 构造 AgentInput
messages = [
    Message(role=Role.USER, content="你好，获取苹果股票价格，和上海的天气！")
]
agent_input = AgentInput(messages=messages)

# 创建 ChatAgent 实例
agent = ChatAgent(
    name="TestChatAgent",
    llm_config={
        "model": chat_agent_config["model"],
        "api_key": chat_agent_config["api_key"],
        "api_base": chat_agent_config["url"],
        **chat_agent_config.get("model_config_dict", {})
    },
    tools=[fetch_weather, get_stock_price]
)


ChatAgent config: {'model': 'deepseek/deepseek-chat', 'api_key': 'sk-2d377ffeddf240dea637617fbd688c27', 'url': 'https://api.deepseek.com', 'model_config_dict': {'temperature': 0.7}}


In [21]:
# 调用step, stream=True
output = agent.step(agent_input, stream=True)
if hasattr(output.content, "__iter__") and not isinstance(output.content, str):
    # 是生成器，逐步打印
    for chunk in output.content:
        print(chunk, end="", flush=True)
    print()
else:
    # 是字符串，直接打印
    print(output.content)

[Executing tools...]


c:\Users\ps\Documents\Academic\Papers\Agents\Auto_research\SciAgents\sciagents\agents\chat_agent.py:275: RuntimeWarning: coroutine 'wait_for' was never awaited
  results.append({
c:\Users\ps\Documents\Academic\Papers\Agents\Auto_research\SciAgents\sciagents\agents\chat_agent.py:275: RuntimeWarning: coroutine 'FunctionTool.execute' was never awaited
  results.append({


目前苹果（AAPL）的股票价格为 **100.2 美元**。  

上海的天气信息暂时无法直接获取，建议您通过以下方式查询实时天气：  
- **中国天气网**：[www.weather.com.cn](http://www.weather.com.cn)  
- **AccuWeather**：[www.accuweather.com](https://www.accuweather.com)  

如果需要其他帮助，请随时告诉我！ 🌟


In [22]:
# 调用step, stream=True
output = agent.step(agent_input, stream=False)
if hasattr(output.content, "__iter__") and not isinstance(output.content, str):
    # 是生成器，逐步打印
    for chunk in output.content:
        print(chunk, end="", flush=True)
    print()
else:
    # 是字符串，直接打印
    print(output.content)

c:\Users\ps\Documents\Academic\Papers\Agents\Auto_research\SciAgents\sciagents\agents\chat_agent.py:275: RuntimeWarning: coroutine 'wait_for' was never awaited
  results.append({
c:\Users\ps\Documents\Academic\Papers\Agents\Auto_research\SciAgents\sciagents\agents\chat_agent.py:275: RuntimeWarning: coroutine 'FunctionTool.execute' was never awaited
  results.append({


目前苹果公司（AAPL）的股票价格为 **100.2 美元**。  

关于上海的天气，由于技术限制我暂时无法直接获取。您可以通过以下方式查询最新天气情况：  
1. **中国天气网**：[www.weather.com.cn](http://www.weather.com.cn)（官方权威）  
2. **AccuWeather**：[上海天气](https://www.accuweather.com)（国际气象服务）  

需要其他金融数据或生活信息吗？我很乐意继续协助！ 📈☀️


In [23]:
# test a_step, stream=True, 异步方法
async def test_step():
    output = await agent.a_step(agent_input, stream=True)
    if hasattr(output.content, "__aiter__"):  # 检查是否为异步生成器
        async for chunk in output.content:
            print(chunk, end="", flush=True)
        print()
    elif hasattr(output.content, "__iter__") and not isinstance(output.content, str):
        for chunk in output.content:
            print(chunk, end="", flush=True)
        print()
    else:
        print(output.content)

await test_step()

[Executing tools...]
### 最新信息更新：
1. **苹果股票(AAPL)**：当前价格为 **100.2 美元**  
2. **上海天气**：今日天气 **晴朗** ☀️  

需要查询其他信息或更详细的天气数据（如温度/湿度）吗？随时告诉我！


In [24]:
# test a_step, stream=False, 异步方法
async def test_step():
    output = await agent.a_step(agent_input, stream=False)
    if hasattr(output.content, "__aiter__"):  # 检查是否为异步生成器
        async for chunk in output.content:
            print(chunk, end="", flush=True)
        print()
    elif hasattr(output.content, "__iter__") and not isinstance(output.content, str):
        for chunk in output.content:
            print(chunk, end="", flush=True)
        print()
    else:
        print(output.content)

await test_step()

### 最新信息更新：
1. **苹果股票(AAPL)**：当前价格为 **100.2 美元**  
2. **上海天气**：今日天气 **晴朗** ☀️  

如果需要更详细的天气信息（如温度、湿度等），或者查询其他股票数据，请随时告诉我！ 😊
